#Pathweigh Analysis.


1. Run all the following cells.
2. Input file is should be named input.csv and can be either microarray or next gen RNA sequence data. Configure the following parameter: is_rnaseq. On Colab an input.csv file can be uploaded to the server.
3. Configure a sample number for export.
4. Use the following table in order to configure a path ID for export:
https://github.com/zurkin1/Pathweigh/blob/master/README.md.

In [ ]:
#Default configuration.
is_rnaseq = False
sample_num = 0
path_id = 708708 #1-and 2-Methylnaphthalene degradation(Kegg).

#Input file should be named input.csv (an example file is provided).
!rm -r Pathweigh
!git clone https://github.com/zurkin1/Pathweigh.git
!mv Pathweigh/code/*.* .

Cloning into 'Pathweigh'...
remote: Enumerating objects: 159, done.
remote: Counting objects: 100% (159/159), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 159 (delta 79), reused 46 (delta 17), pack-reused 0
Receiving objects: 100% (159/159), 12.10 MiB | 11.20 MiB/s, done.
Resolving deltas: 100% (79/79), done.


## Calculate UDP per Gene and Paths Activity.

In [ ]:
from udp import *

udp = calc_udp_multi_process(is_rnaseq = is_rnaseq)

Fri Jul 10 23:06:54 2020 Calculate UDP, is_rnaseq: False
............Fri Jul 10 23:07:32 2020 Done.
Fri Jul 10 23:07:32 2020 Init activity object
Fri Jul 10 23:07:33 2020 Calculate activity and consistency...
...Fri Jul 10 23:07:41 2020 Done.


,sampleID,path_name,pathID,Activity,Consistency,molRole
0,GSM718769,west nile virus(BioCarta),100001,0.701975,0.705708,input
1,GSM718769,west nile virus(BioCarta),100001,0.701975,0.705708,output
2,GSM718769,west nile virus(BioCarta),100001,0.701975,0.705708,agent
3,GSM718769,wnt signaling pathway(BioCarta),100002,0.185143,0.349168,input
4,GSM718769,wnt signaling pathway(BioCarta),100002,0.185143,0.349168,output


In [ ]:
from activity import *

activity_obj = path_activity(udp, is_rnaseq = is_rnaseq)
activity = activity_obj.calc_activity_consistency_multi_process() #Output is saved to output_activity.csv.
activity_obj.activity.head()

##Draw a KGML map for path activity and Download KGML file.

In [ ]:
activity_obj.graphparser(path_id=path_id, sample_num=sample_num)

xml_result = activity_obj.xmlparser(path_id=path_id, sample_num=sample_num)
from bs4 import BeautifulSoup
bs = BeautifulSoup(open('output_path.xml'), "html.parser")
print(bs.prettify())

Create Kegg XML for path: 708708, sample: 0
<?xml version="1.0"?>
<!DOCTYPE pathway SYSTEM "https://www.kegg.jp/kegg/xml/KGML_v0.7.2_.dtd">
<pathway name="path:hsa708708" number="111" org="hsa" title="1- and 2-Methylnaphthalene degradation(Kegg)">
 <entry id="708648" link="https://www.kegg.jp/dbget-bin/www_bget?hsa:4967+hsa:708648" name="hsa:4967 hsa:fm_708637" type="protein">
  <graphics bgcolor="#42ECEF" fgcolor="#FF0000" height="17" name="fm_708637" type="rectangle" width="60" x="1.0">
  </graphics>
 </entry>
 <entry id="708709" link="https://www.kegg.jp/dbget-bin/www_bget?C00158" name="708709" type="reaction">
  <graphics bgcolor="#FFF8F8" fgcolor="#FF0000" height="15" name="reaction" type="circle" width="60" x="1.0" y="1.0">
  </graphics>
 </entry>
 <relation entry1="708648" entry2="708709" type="PPrel">
  <subtype name="activation" value="--&gt;">
  </subtype>
 </relation>
 <entry id="708651" link="https://www.kegg.jp/dbget-bin/www_bget?hsa:4967+hsa:708651" name="hsa:4967 hsa:2-N

##Pathway Assessment.

A pathway can be assessed by differentiating two classed of samples (e.g. cancer vs. normal). A two sample ranksum test is perfomed to evaluate the null hypothesis that pathway scores of class A and class B are sampled from distributions with equal mean and variance. Significant pathways are those for which the null hypothesis is highly unlikely indicating that the pathway behaves differently in these two groups of samples.

In [ ]:
pathname = '1-and 2-Methylnaphthalene degradation(Kegg)
data1 = pd.read_csv('output_activity1.csv')
data1 = data1.loc[data1.path_name == pathname]
data2 = pd.read_csv('output_activity2.csv')
data2 = data2.loc[data2.path_name == pathname]
n1 = len(data1)
n2 = len(data2)
m_u = n1 * n2 * 0.5
stat, p = mann(data1.Activity, data2.Activity)
# sigma_u = np.sqrt(n1*n2*(n1+n2+1)*(10/12))
print(
    f'Stat: {stat}, P-value: {p}, n1: {n1}, n2: {n2}')